In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import numpy as np
import matplotlib.pylab as plt

In [ ]:
# Plot parameters
params = {"mathtext.default": "regular",
          "text.usetex": False,
          "figure.dpi": 300}          
plt.rcParams.update(params)

In [ ]:
from casidm.cascade.cascade_driver import CascadeDriver, InteractionModel

In [ ]:
import chromo
target = chromo.kinematics.CompositeTarget([("N", 0.78), ("O", 0.22)])
ekin = chromo.kinematics.FixedTarget(1e10, "p", target)
model = chromo.models.DpmjetIII193
    
int_model0 = InteractionModel(model, ekin, target)

In [ ]:
from casidm.mceq_utils.mceq_comparison import HybridMCEq
hybrid_mceq = HybridMCEq(pdg_id = 2212,
                           energy = 1e7,
                           theta_deg = 30,
                           slant_depths=[1, 10, 100, 600, 900, 1000, 1195],
                           energy_range=[1e-1, 2e7])

hybrid_mceq.set_result_categories([
                     ("mu", "mu+", "mu-"),
                     ("numu", "numu", "antinumu"),
                     ("nue", "nue", "antinue"),
                     ("pi", "pi+", "pi-"),
                     ("pi0", "pi0"),
                     ("el", "e+", "e-"),
                     ("e+", "e+"),
                     ("e-", "e-")
                     ])

In [ ]:
regular_flux = hybrid_mceq.regular_solution()

In [ ]:
cas_driver = hybrid_mceq.start_cascade_driver(int_model0, 1e2)

In [ ]:
cas_driver.run(100)

In [ ]:
import cProfile

with cProfile.Profile() as pr:
    cas_driver.run(1)
    pr.dump_stats("cas_driver.prof")

In [ ]:
hybrid_flux = hybrid_mceq.hybrid_solution(10, restart = False)

In [ ]:
from casidm.cascade.cascade_analysis import CascadeAnalysis
cascade_analysis = CascadeAnalysis(cas_driver)

In [ ]:
cascade_analysis.plot_ptypes_dist(from_ = 1, to_ = 20, per_run=True)

In [ ]:
line_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure(figsize=(8, 6))
# plt.rcParams['figure.figsize'] = [4, 3]
ixdepth = 0
# plt.stairs(mceq_dist.flux[ixdepth]["pi0"], mceq_dist.e_bins, 
#            label = r"${\mu}^{+} + {\mu}^{-}$ Hybrid" + f"{mceq_dist.slant_depths[ixdepth]}", 
#            linestyle='-',
#            color = line_colors[0])
ixdepths = range(len(hybrid_mceq.slant_depths))
# ixdepths = [4, 5, 6]
for ixdepth in ixdepths:
    plt.stairs(hybrid_flux[ixdepth]["mu"], hybrid_mceq.e_bins, 
            label = r"${\mu}^{+} + {\mu}^{-}$ Hybrid X=" + f"{hybrid_mceq.slant_depths[ixdepth]}g/cm2", 
            linestyle='--',
            color = line_colors[ixdepth])
    
    plt.stairs(regular_flux[ixdepth]["mu"], hybrid_mceq.e_bins, 
            label = r"${\mu}^{+} + {\mu}^{-}$ MCEq X=" + f"{hybrid_mceq.slant_depths[ixdepth]}g/cm2", 
            # linestyle='--',
            color = line_colors[ixdepth])


plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-2, 1e11)
plt.ylim(1e-7, 1e7)
plt.legend(fontsize="7")
plt.grid()
plt.title("\nPrimary particle: proton. " 
          +  r"$E_{tot}=10^{6}$ GeV" + r", $\theta = 30^{\circ}$")
plt.xlabel(r"$E_{kin}$, GeV")
plt.ylabel(r"Counts/bin/primary")

In [ ]:
line_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
ixdepth = 0
# plt.stairs(mceq_dist.flux[ixdepth]["pi0"], mceq_dist.e_bins, 
#            label = r"${\mu}^{+} + {\mu}^{-}$ Hybrid" + f"{mceq_dist.slant_depths[ixdepth]}", 
#            linestyle='-',
#            color = line_colors[0])
ixdepths = range(len(hybrid_mceq.slant_depths))
ixdepths = [4, 5, 6]
for ixdepth in ixdepths:
    # plt.stairs(mceq_dist.flux[ixdepth]["mu"], mceq_dist.e_bins, 
    #         label = r"${\mu}^{+} + {\mu}^{-}$ MCEq" + f"{mceq_dist.slant_depths[ixdepth]}", 
    #         linestyle='--',
    #         color = line_colors[ixdepth])
    
    plt.stairs(hybrid_flux[ixdepth]["mu"]/regular_flux[ixdepth]["mu"], hybrid_mceq.e_bins, 
            label = r"${\mu}^{+} + {\mu}^{-}$, X=" + f"{hybrid_mceq.slant_depths[ixdepth]}g/cm2", 
            # linestyle='--',
            color = line_colors[ixdepth])

plt.xscale("log")
# plt.yscale("log")
plt.xlim(1e-2, 1e5)
plt.ylim(0.5, 1.2)
plt.legend(fontsize="7")
plt.grid()
plt.title("Ratio Hybrid/MCEq"+ "\nPrimary particle: proton. " 
          +  r"$E_{tot}=10^{6}$ GeV" + r", $\theta = 30^{\circ}$")
plt.xlabel(r"$E_{kin}$, GeV")
plt.ylabel(r"Counts/bin/primary")

In [ ]:
from casidm.cascade.cascade_analysis import CascadeAnalysis
cascade_analysis = CascadeAnalysis(cas_driver)
cascade_analysis.print_stats()